# classify_videos

* 適当に分類ルールを作成し歌動画か否かを分類します
* 訓練データを用いて評価を行います

In [1]:
# Pythonの基本ライブラリ
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ファイル操作
import os
import glob

# 機械学習
from transformers import BertJapaneseTokenizer, BertForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.utils import shuffle

# Jupyter上にHTMLを表示する
from IPython.display import HTML

In [3]:
# trainから動画を取得する
file_select = 'okakoro'
# file_select = 'hololive'
file_names = glob.glob('output/'+file_select+'/*')

all_videos = pd.read_csv('output/train/'+file_select+'/videos.csv', index_col=0)
# filtered_videos = filtered_videos.fillna('Nan')
print("全動画数:", all_videos.shape[0])
all_videos.head(2)

全動画数: 44


,Id,Name,ChannelId,Date,Title,Thumbnail,CategoryId,Duration,DurationOriginal,Description,ViewCount,LikeCount,DislikeCount,CommentCount,Label
0,hlwojkAO61A,戌神ころね,UChAnqc_AY5_I3Px5dig3X1Q,2020-08-23T04:38:18Z,早口言葉、噛んだら即終了！,https://i.ytimg.com/vi/hlwojkAO61A/hqdefault.jpg,24,359,PT5M59S,わん！\n\n前回→https://www.youtube.com/watch?v=BaYx...,145488,8448,0,202,0
1,3uZof09cUlA,戌神ころね,UChAnqc_AY5_I3Px5dig3X1Q,2021-08-24T13:46:23Z,8位以下なら即終了・リベンジのリベンジ・エターナル・ファイナルラスト,https://i.ytimg.com/vi/3uZof09cUlA/hqdefault.jpg,20,375,PT6M15S,Play Game : Mario Kart 8 Deluxe ( switch )\nタグ...,118973,6120,0,313,0


In [4]:
# 歌動画一覧を渡しフィルタの性能を評価
def validation_filter(filtered_videos, all_videos):
    pred_1 = filtered_videos.shape[0]
    true_positive = filtered_videos[filtered_videos['Label']==1].shape[0]
    false_positive = pred_1 - true_positive
    
    # 歌動画と予測されなかった動画を取り出す
    all_ids = np.array(all_videos['Id'])
    filtered_ids = np.array(filtered_videos['Id'])
    not_filtered_ids = np.zeros(all_ids.size)
    for i, ids in enumerate(all_ids):
        if ids not in filtered_ids:
            not_filtered_ids[i] = 1
    not_filtered_ids = not_filtered_ids.astype(bool)
    not_filtered_videos = all_videos[not_filtered_ids]
    
    pred_0 = not_filtered_videos.shape[0]
    true_negative = not_filtered_videos[not_filtered_videos['Label']==0].shape[0]
    false_negative = pred_0 - true_negative
    # print(true_positive, false_positive, true_negative, false_negative)
    return (true_positive + true_negative) / all_videos.shape[0]

## ルールベースで分類
以下の基準でルールを作り適当に分類
* カテゴリによるフィルタリング
* 単語によるフィルタリング
* タグによるフィルタリング

In [6]:
######################################
# フィルタリング1: カテゴリにてフィルタリング
######################################

# 音楽、エンターテイメント、ゲーム動画以外を除外（何故かゲーム動画の中にも多く含まれていた）
# https://so-zou.jp/web-app/tech/web-api/google/youtube/category.htm

n = all_videos.shape[0]
filtered_ids = np.zeros(n)
categories = np.array(all_videos['CategoryId'])
titles = np.array(all_videos['Title'])
descs = np.array(all_videos['Description'])

for i, category in enumerate(categories):
    if category not in [10, 20, 24]:
        filtered_ids[i] -= 10000

######################################
# フィルタリング2: Title、Descriptionによるフィルタリング
######################################

# titleに"歌ってみた"などが入る動画は高い可能性で歌動画と判断
title_posiwords = [
    '歌ってみた', 'オリジナル', 'ORIGINAL', 'COVER', 'FEAT'
]
for i, title in enumerate(titles):
    title_upper = title.upper()
    for keyword in title_posiwords:
        if keyword in title_upper:
            filtered_ids[i] += 100

# 逆に"実況"や"アニメ"など入っている場合は歌動画ではないと判断
title_negawords = [
    '実況', 'クロスフェード', 'アニメ', 'アカペラ'
]
for i, title in enumerate(titles):
    title_upper = title.upper()
    for keyword in title_negawords:
        if keyword in title_upper:
            filtered_ids[i] -= 100

# descriptionに"歌ってみた"などが入る動画は高い可能性で歌動画と判断
desc_keywords = [
    '歌ってみた', '歌詞', '作詞', '作曲', '編曲', '原曲', '本家', '歌唱', 'ボーカル', 'ミックス', 
    'SONG', 'MUSIC', 'VOCAL', 'MIX', 'COMPOSER', 'LYRIC', '歌', '曲'
]
for i, desc in enumerate(descs):
    desc_upper = desc.upper()
    for keyword in desc_keywords:
        if keyword in desc_upper:
            filtered_ids[i] += 1

filetered_videos = all_videos[filtered_ids > 1]
print("フィルタリング後のデータ数：", filetered_videos.shape[0])
accuracy = validation_filter(filetered_videos, all_videos)
print('Accuracy:', accuracy)

フィルタリング後のデータ数： 28
Accuracy: 0.5909090909090909


In [27]:
# Tagはほぼ機能していないため除外
######################################
# フィルタリング3: Tagによるフィルタリング
######################################

# n = all_videos.shape[0]
# tags = np.array(all_videos['Tags'])
# filtered_ids = np.zeros(n)

# for i, tag in enumerate(tags):
#     tag = tag.replace('[', '')
#     tag = tag.replace(']', '')
#     tag = tag.replace('\'', '')
#     taglist = tag.split(', ')
    
#     if '歌ってみた' in taglist:
#         filtered_ids[i] = 1

# filetered_videos = all_videos[filtered_ids > 0]
# print("フィルタリング後のデータ数：", filetered_videos.shape[0])
# accuracy = validation_filter(filetered_videos, all_videos)
# print('Accuracy:', accuracy)

## LASSOを用いて分類

In [7]:
######################################
# フィルタリング4: Title、Descriptionをone-hotベクトルに変換してLASSOで分類
######################################

# トークナイザの準備
MODEL_NAME = 'cl-tohoku/bert-base-japanese-whole-word-masking'
tokenizer = BertJapaneseTokenizer.from_pretrained(MODEL_NAME)
# print(bert_sc.config) # 語彙数が32000であることを確認

In [8]:
# 単語分割

# TitleとDescriptionの両方で作成
max_length = 256 # 最大で512
X, Y = [], []

# DataFrameからrow毎に取り出す
# input_idsのみで十分
for i, row in all_videos.iterrows():
    # text = row['Title']
    text = row['Description']
    # text = row['Title'] + row['Description']
    
    encoding = tokenizer(
        text,
        max_length=max_length, 
        padding='max_length',
        truncation=True
    )
    
    input_ids = encoding['input_ids']
    input_ids = np.unique(input_ids) # countは一旦無視してuniqueにする
    input_ids = input_ids[input_ids>=5] # [PAD], [UNK], [CLS], [SEP], [MASK]を削除
    
    zeros = np.zeros(32000)
    zeros[input_ids] = 1
    zeros = np.append(zeros, i) # IDを可視化するための一時処理
    X.append(zeros)
    Y.append(row['Label'])

X = np.array(X)
Y = np.array(Y)

In [9]:
# CVで正解率を評価
def cross_validation(X, Y, k=5):
    # XとYをシャッフル
    X, Y = shuffle(X, Y, random_state=0)
    
    # XとYをk分割
    n = X.shape[0]
    X_devs, Y_devs = [], []
    for i in range(k):
        if i != k-1:
            X_dev, Y_dev = X[i*(n//5):(i+1)*(n//5)], Y[i*(n//5):(i+1)*(n//5)]
        else:
            X_dev, Y_dev = X[i*(n//5):], Y[i*(n//5):]
        X_devs.append(X_dev)
        Y_devs.append(Y_dev)
        
    # 1つをvalidation, 1つをテストとしてテスト誤差を計算する
    test_accuracy = 0
    for i in range(k):
        print('k-cross-validation :', i+1, '/', k)
        X_train_tmp, Y_train_tmp = [], []
        for j in range(k-2):
            X_train_tmp.append(X_devs[(i+j)%k])
            Y_train_tmp.append(Y_devs[(i+j)%k])
        X_train = np.concatenate(X_train_tmp)
        Y_train = np.concatenate(Y_train_tmp)
        X_val, Y_val = X_devs[(i+k-2)%k], Y_devs[(i+k-2)%k]
        X_test, Y_test = X_devs[(i+k-1)%k], Y_devs[(i+k-1)%k]
        
        # logscaleでハイパラの候補を準備
        params = np.logspace(-2, 0)
        val_best = 0
        test_best = 0
        for param in params:
            lr = LogisticRegression(penalty='l1', solver='liblinear', C=param)
            lr.fit(X_train, Y_train)
            
            Y_pred = lr.predict(X_val)
            if np.sum(Y_pred==Y_val) > val_best:
                val_best = np.sum(Y_pred==Y_val)
                # Accuracyも更新する
                Y_pred = lr.predict(X_test)
                test_best = np.sum(Y_pred==Y_test)
            
        test_accuracy += test_best    

    return test_accuracy/n

In [10]:
# CVを実施
print('CV Accuracy', cross_validation(X, Y))

k-cross-validation : 1 / 5
k-cross-validation : 2 / 5
k-cross-validation : 3 / 5
k-cross-validation : 4 / 5
k-cross-validation : 5 / 5
CV Accuracy 0.7272727272727273


In [12]:
# 頻度が高い単語を取り出す
freq_word_ids = []
for label in range(2):
    freq_word_id = np.sum(X[Y == label], axis=0)
    freq_word_ids.append(freq_word_id)
    freq_word_rank = np.argsort(freq_word_id)[::-1]
    # print(tokenizer.convert_ids_to_tokens(freq_word_rank[0:20]))

# 特定のクラスでのみ頻度が高い単語を取り出す
spec_word_ids = freq_word_ids[1] - freq_word_ids[0]
spec_word_rank = np.argsort(spec_word_ids)[::-1]
print(tokenizer.convert_ids_to_tokens(spec_word_rank[:20]))
print(tokenizer.convert_ids_to_tokens(spec_word_rank[-20:]))

['[UNK]', '様', '##@', '##u', '##又', 'ne', '_', 'Mi', 'おか', '猫', '##ゆ', '##ta', '##x', '##l', '本家', '##ko', '##ie', 'Mov', '##r', '##ay']
['Mar', 'Game', 'ころ', 'Play', '##えもん', '##itch', '##am', '→', '##itter', '##io', 'tw', 'Twitter', '生', '##iko', '##ug', '#', '##神', '##e', 'in', '##ron']


In [13]:
# 全データを使って学習し、パラメータの重みから重要な単語を取り出す
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 0)
# X_train, Y_train = shuffle(X, Y, random_state=0)

# IDを可視化するための一時処理
X_train = X_train.T[0:32000].T

# ロジスティック回帰分析
lr = LogisticRegression(penalty='l1', solver='liblinear', C=1.0) # ロジスティック回帰モデルのインスタンスを作成
lr.fit(X_train, Y_train) # ロジスティック回帰モデルの重みを学習

# 重みを確認
w = lr.coef_[0]
print(np.sort(w)[0:5])
print(tokenizer.convert_ids_to_tokens(np.argsort(w)[0:5]))

print(np.sort(w)[::-1][0:5])
print(tokenizer.convert_ids_to_tokens(np.argsort(w)[::-1][0:5]))

# 予測
# IDを可視化するための一時処理
# X_test = X_test.T[0:32000].T
# Y_pred = lr.predict(X_test)
# print(np.sum(Y_pred==Y_test), '/', Y_test.shape[0])

[-0.84059914 -0.64827974 -0.56814835 -0.20505461 -0.15746338]
['##e', '##ok', '##w', ')', 'な']
[2.47054917 0.94155783 0.         0.         0.        ]
['オリジナル', '曲', '##巽', '日程', 'マーティン']


In [15]:
def output_html_test(videos, predict, idxs, top_n=10):
    html = '<h1>動画一覧を表示</h1>'
    html += '<div style="float:left;">'
    
    label0_count, label1_count = 0, 0
    label0_html, label1_html = '', ''

    for i in range(len(predict)):
        idx = idxs[i]
        label = int(predict[i])
        if label == 0:
            if label0_count < top_n:
                if (label0_count) % (top_n/2) == 0:
                    label0_html += '<br>'
                label0_html += ('<img src="'+np.array(videos['Thumbnail'])[idx] +' "alt="取得できませんでした" width="150">')
                label0_count += 1
        elif label == 1:
            if label1_count < top_n:
                if (label1_count) % (top_n/2) == 0:
                    label1_html += '<br>'
                label1_html += ('<img src="'+np.array(videos['Thumbnail'])[idx] +' "alt="取得できませんでした" width="150">')
                label1_count += 1
        if label0_count == top_n and label1_count == top_n:
            break

    html += ('<h2>非歌動画</h2>' + label0_html + '<br>')
    html += ('<h2>歌動画</h2>' + label1_html + '<br>')
    html += '</div>'
    return html

In [17]:
# 良かった方法でランキングにして可視化

# IDを可視化するための一時処理
video_ids = X_test.T[32000].astype(int)
X_test_ = X_test.T[0:32000].T
Y_pred = lr.predict(X_test_)

HTML(output_html_test(all_videos, Y_pred, video_ids, top_n=10))